# Projeto 1 - Ciência dos Dados

Nome: Venâncio Freitas de Araújo Filho

Nome: WELLINGTON RODRIGUES DA SILVA

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\venan\OneDrive - Insper - Institudo de Ensino e Pesquisa\Materias 2° semestre Insper\Ciência de dados\Projeto 1


Dados de Treino: 

Carregando a base de dados com os tweets classificados manualmente:

In [3]:
train = pd.read_excel('dados_treino.xlsx')
train

,Mensagem,Target
0,Muito erros grosseiros de tradução. O material...,Editora
1,"Primeiramente meu pedido nem veio,Mas me reemb...",Amazon
2,"Livro com um monte de clichês, mas com pouco c...",Editora
3,Sim.adorei o livro Vou continuar a comprar o r...,Editora
4,Não recebi o livro . Não sei o motivo. Foi fei...,Amazon
...,...,...
293,O livro foi classificado como novo no site e c...,Amazon
294,"Não gostei porquê a capa veio com um rasgo, o ...",Amazon
295,EU NEM SE QUER RECEBI O PRODUTO! Voces são mui...,Amazon
296,Não consigo entender o alto valor desses e-boo...,Amazon


In [4]:
train.Target.value_counts(True).round(3)

Autor      0.377
Editora    0.311
Amazon     0.311
Name: Target, dtype: float64

Dados de Teste


In [5]:
test = pd.read_excel('dados_teste.xlsx')
test.head(5)

,Mensagem,Target
0,Amontoado de proposições absurdas destinadas a...,Autor
1,Quando comprei esse livro fui com uma grande e...,Autor
2,Li um terço do livro. Arrastei-me para procura...,Autor
3,"Sensacionalista, raso e cheio de distorções ma...",Autor
4,A mensagem do livro da pra entender em 1 págin...,Autor


In [6]:
test.Target.value_counts()

Autor      102
Editora     76
Amazon      33
Name: Target, dtype: int64

In [7]:
test.Target

0      Autor
1      Autor
2      Autor
3      Autor
4      Autor
       ...  
206    Autor
207    Autor
208    Autor
209    Autor
210    Autor
Name: Target, Length: 211, dtype: object

___
## Classificador automático


Faça aqui uma descrição do seu assunto e o que considerou como relevante ou não relevante na classificação dos tweets (Target).

ESCREVA AQUI...

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

## 1.Vamos Iniciar Extraindo os Dados Referente à cada rótulo

Função CleanUp:

In [8]:
import re 

def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[´"!-.:?;$'']' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

1. Dados do Autor 

In [9]:
dados_autor  = train.loc[train.Target == 'Autor' , :]
print(dados_autor.head(2))

                                             Mensagem Target
40  Como o livro se concentra na política american...  Autor
42  Achei ridículo! Eu realmente esperava uma hist...  Autor


In [10]:
#Montando um Texto
texto_comentários_autor = ''
for i in dados_autor.Mensagem: 
    texto_comentários_autor += i
    texto_comentários_autor += ' '
texto_comentários_autor = cleanup(texto_comentários_autor)
texto_comentarios_autor = texto_comentários_autor.lower()
texto_comentarios_autor = texto_comentarios_autor.split(' ')
tabela_autor = pd.Series(texto_comentarios_autor )
tabela_autor.head()
print(tabela_autor.shape)

(6683,)


Tabelas Relativas ao autor 

In [11]:
#Frequências Absolutas 
tabela_autor_abs = tabela_autor.value_counts()

In [12]:
#Frequências relativas 
tabela_autor_relativa = tabela_autor.value_counts(True)
tabela_autor_relativa 

de                 0.038306
o                  0.035463
que                0.033368
e                  0.030226
a                  0.029328
                     ...   
passaria           0.000150
praticamente       0.000150
apontar            0.000150
efetivo            0.000150
relacionamentos    0.000150
Length: 1982, dtype: float64

In [13]:
tabela_autor_abs.sum()

6683

## 2. Dados da Editora 

In [14]:
dados_editora = train.loc[train.Target == 'Editora' , :]
dados_editora.shape

(90, 2)

In [15]:
comentarios_editora = dados_editora.Mensagem
texto_editora = ''
for e in comentarios_editora:
    texto_editora += e 
    texto_editora += ' '
texto_editora = cleanup(texto_editora)
texto_editora = texto_editora.lower()
texto_editora = texto_editora.split(' ')
tabela_editora = pd.Series(texto_editora)
print(tabela_editora.head())
print(tabela_editora.shape)

0         muito
1         erros
2    grosseiros
3            de
4      tradução
dtype: object
(4727,)


Frequências relativas às tabelas

In [16]:
#Frequência absoluta
tabela_editora_abs = tabela_editora.value_counts()

In [17]:
#Frequência Relativa 
tabela_editora_relativa = tabela_editora.value_counts(True)
tabela_editora_relativa

de          0.041041
a           0.033425
o           0.029406
que         0.028348
e           0.027925
              ...   
perdemos    0.000212
lendo       0.000212
viu         0.000212
presente    0.000212
frustada    0.000212
Length: 1617, dtype: float64

In [18]:
#Quantidade de palavras 
tabela_editora_abs.sum()

4727

3. Dados da Amazon


In [19]:
dados_amazon = train.loc[train.Target == 'Amazon' , :]
print(dados_amazon)
print(dados_amazon.shape)

                                              Mensagem  Target
1    Primeiramente meu pedido nem veio,Mas me reemb...  Amazon
4    Não recebi o livro . Não sei o motivo. Foi fei...  Amazon
7    Pelo amor de Deus!!!! que preço é este......só...  Amazon
12   FiZ a compra errada, era pra comprar o livro e...  Amazon
23   Não recebi o livro... Acredito que tenham envi...  Amazon
..                                                 ...     ...
293  O livro foi classificado como novo no site e c...  Amazon
294  Não gostei porquê a capa veio com um rasgo, o ...  Amazon
295  EU NEM SE QUER RECEBI O PRODUTO! Voces são mui...  Amazon
296  Não consigo entender o alto valor desses e-boo...  Amazon
297  a caixa veio rasgada e amassada, gostaria de u...  Amazon

[90 rows x 2 columns]
(90, 2)


In [20]:
comentarios_amazon = dados_amazon.Mensagem
texto_amazon = ''
for texto in comentarios_amazon:
    texto_amazon += texto 
    texto_amazon += ' '
texto_amazon = cleanup(texto_amazon)
texto_amazon = texto_amazon.lower()
texto_amazon = texto_amazon.split(' ')
tabela_amazon = pd.Series(texto_amazon)
print(tabela_amazon)

0       primeiramente
1                 meu
2              pedido
3                 nem
4             veiomas
            ...      
4686            estão
4687               em
4688            otimo
4689           estado
4690                 
Length: 4691, dtype: object


Frequências referentes à Amazon: 

In [21]:
#Frequência absoluta 
tabela_amazon_abs = tabela_amazon.value_counts()

In [22]:
#Frequência relativa
tabela_amazon_relativa = tabela_amazon.value_counts(True)
tabela_amazon_relativa

o          0.037732
a          0.033468
e          0.030484
que        0.027499
de         0.022170
             ...   
olá        0.000213
cópia      0.000213
enviada    0.000213
lamppm     0.000213
otimo      0.000213
Length: 1369, dtype: float64

In [23]:
tabela_amazon_abs.sum()

4691

___
### Construindo o Classificador a partir dos testes

Agora você deve testar o seu classificador com a base de Testes.

In [24]:
todas_palavras = texto_editora + texto_amazon + texto_comentarios_autor

# 1 Probabilidade de Cada Rótulo dentro do texto

In [25]:
P_Autor = sum(tabela_autor_abs) / len(todas_palavras)
P_Autor

0.41506738711881247

In [26]:
P_Amazon = sum(tabela_amazon_abs) / len(todas_palavras)
P_Amazon

0.291348363455686

In [27]:
P_Editora = sum(tabela_editora_abs) / len(todas_palavras)
P_Editora

0.29358424942550154

# 2. Probabilidade de aparecer em cada frase

In [28]:
P_frase_dado_autor = 1
P_frase_dado_editora = 1
P_frase_dado_amazon = 1
lista_teste = []
i = 0
lista_probs = []

In [33]:
for frase in test.Mensagem:
    frase = cleanup(str(frase))
    frase = frase.lower()
    frase = frase.split()
    
    for palavras in frase:
        
        if palavras in texto_comentarios_autor: 
            P_frase_dado_autor *= tabela_autor_relativa[palavras] + (1/len(tabela_autor))
        else: 
            P_frase_dado_autor *= (1/len(tabela_autor))
            
        
        if palavras in texto_amazon: 
            P_frase_dado_amazon *= tabela_amazon_relativa[palavras] + (1/len(tabela_amazon))
        else: 
            P_frase_dado_amazon *= (1/len(tabela_amazon))
        
                
        if palavras in texto_editora: 
            P_frase_dado_editora *= tabela_editora_relativa[palavras] + (1/len(tabela_editora))
        else: 
            P_frase_dado_editora *= (1/len(tabela_editora))
            
        
            
    P_autor_dado_frase = P_frase_dado_autor * P_Autor
    lista_probs.append( P_autor_dado_frase)
    
    P_editora_dado_frase = P_frase_dado_editora * P_Editora
    lista_probs.append( P_editora_dado_frase)
    
    P_amazon_dado_frase =  P_frase_dado_amazon * P_Amazon
    lista_probs.append( P_amazon_dado_frase)
    
    if max(lista_probs) == lista_probs[0]:
        lista_teste.append('Autor')
    elif max(lista_probs) == lista_probs[1]:
        lista_teste.append('Editora')
    elif max(lista_probs) == lista_probs[2]:
        lista_teste.append('Amazon')
        
    P_frase_dado_editora = 1
    P_frase_dado_amazon = 1
    P_frase_dado_autor = 1
    
    P_autor_dado_frase = 1
    
    P_editora_dado_frase = 1
    
    P_amazon_dado_frase =  1
    lista_probs = []
    
    i += 1
    
lista_teste

['Editora',
 'Autor',
 'Editora',
 'Editora',
 'Editora',
 'Editora',
 'Editora',
 'Amazon',
 'Amazon',
 'Autor',
 'Editora',
 'Autor',
 'Amazon',
 'Autor',
 'Autor',
 'Autor',
 'Editora',
 'Editora',
 'Autor',
 'Editora',
 'Editora',
 'Editora',
 'Autor',
 'Autor',
 'Editora',
 'Amazon',
 'Editora',
 'Amazon',
 'Editora',
 'Editora',
 'Autor',
 'Editora',
 'Autor',
 'Autor',
 'Editora',
 'Autor',
 'Editora',
 'Autor',
 'Editora',
 'Editora',
 'Autor',
 'Autor',
 'Amazon',
 'Editora',
 'Autor',
 'Autor',
 'Amazon',
 'Autor',
 'Autor',
 'Amazon',
 'Amazon',
 'Autor',
 'Editora',
 'Autor',
 'Editora',
 'Editora',
 'Autor',
 'Autor',
 'Amazon',
 'Autor',
 'Amazon',
 'Amazon',
 'Editora',
 'Autor',
 'Amazon',
 'Autor',
 'Amazon',
 'Autor',
 'Amazon',
 'Autor',
 'Editora',
 'Amazon',
 'Amazon',
 'Editora',
 'Amazon',
 'Amazon',
 'Amazon',
 'Autor',
 'Editora',
 'Editora',
 'Amazon',
 'Autor',
 'Amazon',
 'Editora',
 'Autor',
 'Autor',
 'Amazon',
 'Autor',
 'Editora',
 'Editora',
 'Autor',
 

## 4. Agora, Vamos Verificar a Performance do Treino

# Verificando os verdadeiros positivos

In [30]:
test['Bot'] = lista_teste
tabela_verificação = (pd.crosstab(test.Target , test.Bot , normalize = True) * 100).round(3)
tabela_verificação

Bot,Amazon,Autor,Editora
Target,,,
Amazon,14.692,0.000,0.948
Autor,3.791,25.118,19.431
Editora,7.109,12.322,16.588


# Verificando Acertos no Rótulo Autor 


# Verificando Acertos no Rótulo Editora

## Verificando Acertos no Rótulo Amazon

## Acurácia 

## 5. Agora, Vamos calcular os desvios

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CONSIDEROU mais de duas categorias na variável Target e INCREMENTOU a quantidade de notícias, mantendo pelo menos 250 notícias por categoria (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* Para Target com duas categorias: CRIOU pelo menos quatro categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das Notícias entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**

In [32]:
if P_autor_dado_frase >= P_editora_dado_frase and P_autor_dado_frase >= P_amazon_dado_frase:
        dicio_treino[i] = 'Autor'
        
    elif P_editora_dado_frase >= P_autor_dado_frase and P_editora_dado_frase >= P_amazon_dado_frase:
        dicio_treino[i] = 'Editora'
        
    elif P_amazon_dado_frase > P_autor_dado_frase and P_amazon_dado_frase > P_editora_dado_frase: 
        dicio_treino[i] = 'Amazon'
    

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 4)